### Installing the necessary requirements

In [2]:
!pip install nltk | tail -n 1
!pip install sentence_transformers | tail -n 1
!pip install chromadb==0.3.27 | tail -n 1
!pip install "ibm-watson-machine-learning>=1.0.312" | tail -n 1


[notice] A new release of pip is available: 23.0 -> 23.2.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0 -> 23.2.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0 -> 23.2.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0 -> 23.2.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [77]:
from platform import python_version
python_version()

'3.9.6'

### Importing the data and initializing the model related params

In [5]:
import os, types
import pandas as pd
#from botocore.client import Config
#import ibm_boto3
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.api.types import EmbeddingFunction

def __iter__(self): return 0

# Adding the default params to the models 

api_key = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
project_id = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'

model_credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
}


'''cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id= api_key,
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us-south.cloud-object-storage.appdomain.cloud')

bucket = 'watsonxchallengesandbox-donotdelete-pr-hdmgfjqdjzh8c0'
'''
object_key = 'vulnerability.csv'
object_key = 'Vulners-3000.xlsx'


'''body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )
'''

df = pd.read_excel(object_key)
df.head()

/Users/rakeshpolepeddi/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,ID,title,description
0,0,RHBA-2011:1656: mod_nss bug fix update (None),The mod_nss module provides strong cryptograph...
1,1,RHBA-2012:0763: glibc bug fix and enhancement ...,The glibc packages provide the standard C and ...
2,2,RHBA-2012:0881: freeradius bug fix and enhance...,FreeRADIUS is an open-source Remote Authentica...
3,3,RHBA-2013:0363: sudo bug fix and enhancement u...,The sudo (super user do) utility allows system...
4,4,RHBA-2013:0386: tuned bug fix update (None),The tuned packages contain a daemon that tunes...


In [67]:
df.describe()

,ID
count,3001.000000
mean,1500.000000
std,866.458404
min,0.000000
25%,750.000000
50%,1500.000000
75%,2250.000000
max,3000.000000


### Create an embedding function

Note that you can feed a custom embedding function to be used by chromadb. The performance of chromadb may differ depending on the embedding model used.

In [6]:
class MiniLML6V2EmbeddingFunction(EmbeddingFunction):
    MODEL = SentenceTransformer('all-MiniLM-L6-v2')
    def __call__(self, texts):
        return MiniLML6V2EmbeddingFunction.MODEL.encode(texts).tolist()
emb_func = MiniLML6V2EmbeddingFunction()

### Set up Chroma upsert

Upserting a document means update the document even if it exists in the database. Otherwise re-inserting a document throws an error. This is useful for experimentation purpose.

In [10]:
data_root = "data"
knowledge_base_dir = f"{data_root}/knowledge_base"
class ChromaWithUpsert:
    def __init__(
            self,
            name = "watsonx_rag_collection",
            persist_directory=knowledge_base_dir,
            embedding_function=None,
            collection_metadata = None,
    ):
        self._client_settings = chromadb.config.Settings()
        if persist_directory is not None:
            self._client_settings = chromadb.config.Settings(
                chroma_db_impl="duckdb+parquet",
                persist_directory=persist_directory,
            )
        self._client = chromadb.Client(self._client_settings)
        self._embedding_function = embedding_function
        self._persist_directory = persist_directory
        self._name = name
        self._collection = self._client.get_or_create_collection(
            name=self._name,
            embedding_function=self._embedding_function
            if self._embedding_function is not None
            else None,
            metadata=collection_metadata,
        )

    def upsert_texts(
        self,
        texts,
        metadata = None,
        ids = None,
        **kwargs,
    ):
        """Run more texts through the embeddings and add to the vectorstore.
        Args:
            :param texts (Iterable[str]): Texts to add to the vectorstore.
            :param metadatas (Optional[List[dict]], optional): Optional list of metadatas.
            :param ids (Optional[List[str]], optional): Optional list of IDs.
            :param metadata: Optional[List[dict]] - optional metadata (such as title, etc.)
        Returns:
            List[str]: List of IDs of the added texts.
        """
        # TODO: Handle the case where the user doesn't provide ids on the Collection
        if ids is None:
            import uuid
            ids = [str(uuid.uuid4()) for _ in range(0, len(texts))]
        embeddings = None
        self._collection.upsert(
            metadatas=metadata, documents=texts, ids=ids
        )
        return ids

    def is_empty(self):
        return self._collection.count()==0

    def persist(self):
        self._client.persist()

    def query(self, query_texts:str, n_results:int=5):
        """
        Returns the closests vector to the question vector
        :param query_texts: the question
        :param n_results: number of results to generate
        :return: the closest result to the given question
        """
        return self._collection.query(query_texts=query_texts, n_results=n_results)

### Embed and index documents with Chroma

**Note: Could take several minutes if you don't have pre-built indices**

In [70]:
%%time
df['indextext'] = df['title'].astype(str) + "\n" + df['description']



chroma = ChromaWithUpsert(
    name=f"nq910_minilm6v2",
    embedding_function=emb_func,  # you can have something here using /embed endpoint
    persist_directory=knowledge_base_dir,
)
print(chroma.is_empty())
if chroma.is_empty():
    _ = chroma.upsert_texts(
        texts=df.indextext.tolist(),
        metadata=[{'title': title} for title in df.title], ids=None,)
    chroma.persist()

True
CPU times: user 2min 33s, sys: 29.2 s, total: 3min 2s
Wall time: 46.5 s


### Ask question based on embeddings

In [23]:
chroma = ChromaWithUpsert(
    name=f"nq910_minilm6v2",
    embedding_function=emb_func,  # you can have something here using /embed endpoint
    persist_directory=knowledge_base_dir,
)


relevant_contexts = []
question_texts = ['Is mod_nss safe to use?']
for question_text in question_texts:
    relevant_chunks = chroma.query(
        query_texts=[question_text],
        n_results=2,
    )
    relevant_contexts.append(relevant_chunks)

Get the set of chunks for one of the questions.

In [24]:
sample_chunks = relevant_contexts[0]
for i, chunk in enumerate(sample_chunks['documents'][0]):
    print("=========")
    print("Paragraph index : ", sample_chunks['ids'][0][i])
    print("Paragraph : ", chunk)
    print("Distance : ", sample_chunks['distances'][0][i])

Paragraph index :  f155b938-bc59-4498-9817-aceda4b28fcb
Paragraph :  RHSA-2016:2602: mod_nss security, bug fix, and enhancement update (Low)
The mod_nss module provides strong cryptography for the Apache HTTP Server via the Secure Sockets Layer (SSL) and Transport Layer Security (TLS) protocols, using the Network Security Services (NSS) security library.

The following packages have been upgraded to a newer upstream version: mod_nss (1.0.14). (BZ#1299063)

Security Fix(es):

* A flaw was found in the way mod_nss parsed certain OpenSSL-style cipher strings. As a result, mod_nss could potentially use ciphers that were not intended to be enabled. (CVE-2016-3099)

This issue was discovered by Rob Crittenden (Red Hat).

Additional Changes:

For detailed information on changes in this release, see the Red Hat Enterprise Linux 7.3 Release Notes linked from the References section.
Distance :  0.9599862694740295
Paragraph index :  ef77c233-1054-46d1-8358-94c0aa3bd665
Paragraph :  RHSA-2013:1779

In [25]:
def make_prompt(context, question_text):
    return (f"Please answer the following.\n"
          + f"{context}:\n\n"
          + f"{question_text}")

prompt_texts = []

for relevant_context, question_text in zip(relevant_contexts, question_texts):
    context = "\n\n\n".join(relevant_context["documents"][0])
    prompt_text = make_prompt(context, question_text)
    prompt_texts.append(prompt_text)

print(prompt_texts[0])

Please answer the following.
RHSA-2016:2602: mod_nss security, bug fix, and enhancement update (Low)
The mod_nss module provides strong cryptography for the Apache HTTP Server via the Secure Sockets Layer (SSL) and Transport Layer Security (TLS) protocols, using the Network Security Services (NSS) security library.

The following packages have been upgraded to a newer upstream version: mod_nss (1.0.14). (BZ#1299063)

Security Fix(es):

* A flaw was found in the way mod_nss parsed certain OpenSSL-style cipher strings. As a result, mod_nss could potentially use ciphers that were not intended to be enabled. (CVE-2016-3099)

This issue was discovered by Rob Crittenden (Red Hat).

Additional Changes:

For detailed information on changes in this release, see the Red Hat Enterprise Linux 7.3 Release Notes linked from the References section.


RHSA-2013:1779: mod_nss security update (Moderate)
The mod_nss module provides strong cryptography for the Apache HTTP Server
via the Secure Sockets Lay

### Adding the model params here

In [17]:
model_id = ModelTypes.FLAN_UL2

def create_model(params):
    return Model(model_id=model_id, params=parameters, credentials=model_credentials, project_id=project_id)

In [28]:
from ibm_watson_machine_learning.foundation_models.utils.enums import DecodingMethods
parameters = {
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY,
    GenParams.MIN_NEW_TOKENS: 50,
    GenParams.MAX_NEW_TOKENS: 100
}
model = create_model(parameters)
results = []

for prompt_text in prompt_texts:
    results.append(model.generate_text(prompt=prompt_text))

print(results)

['no, mod_nss could potentially use ciphers that were not intended to be enabled. (CVE-2016-3099) This issue was discovered by Rob Crittenden (Red Hat). Additional Changes: For detailed information on changes in this release, see the Red Hat Enterprise Linux 7.3 Release Notes linked from the References section.']


### Function to create and generate data from prompts.

In [76]:
def run_prompts(comments, instruction=None):
    parameters = {
        GenParams.MAX_NEW_TOKENS: 100
    }
    model = create_model(params)
    results = []
    for input_text in comments:
        results.append({'Q': input_text, 'A': model.generate_text(prompt=" ".join([instruction, input_text] if instruction else [input_text]))})
    return results

In [7]:
comments = ['what is the need to update java', 'Is there any open vulnerability in java', 'Is there any fix for vulnerability in java 1.7']
instruction = '''

Title: RHSA-2013:0751: java-1.7.0-openjdk security update (Critical)

packages provide the OpenJDK 7 Java Runtime Environment and the
OpenJDK 7 Software Development Kit.

Multiple flaws were discovered in the font layout engine in the 2D
component. An untrusted Java application or applet could possibly use these
flaws to trigger Java Virtual Machine memory corruption. (CVE-2013-1569,
CVE-2013-2383, CVE-2013-2384)

Multiple improper permission check issues were discovered in the Beans,
Libraries, JAXP, and RMI components in OpenJDK. An untrusted Java
application or applet could use these flaws to bypass Java sandbox
restrictions. (CVE-2013-1558, CVE-2013-2422, CVE-2013-2436, CVE-2013-1518,
CVE-2013-1557)

The previous default value of the java.rmi.server.useCodebaseOnly property
permitted the RMI implementation to automatically load classes from
remotely specified locations. An attacker able to connect to an application
using RMI could use this flaw to make the application execute arbitrary
code. (CVE-2013-1537)

Note: The fix for CVE-2013-1537 changes the default value of the property
to true, restricting class loading to the local CLASSPATH and locations
specified in the java.rmi.server.codebase property. Refer to Red Hat
Bugzilla bug 952387 for additional details.

The 2D component did not properly process certain images. An untrusted Java
application or applet could possibly use this flaw to trigger Java Virtual
Machine memory corruption. (CVE-2013-2420)

It was discovered that the Hotspot component did not properly handle
certain intrinsic frames, and did not correctly perform access checks and
MethodHandle lookups. An untrusted Java application or applet could
use these flaws to bypass Java sandbox restrictions. (CVE-2013-2431,
CVE-2013-2421, CVE-2013-2423)

It was discovered that JPEGImageReader and JPEGImageWriter in the ImageIO
component did not protect against modification of their state while
performing certain native code operations. An untrusted Java application or
applet could possibly use these flaws to trigger Java Virtual Machine
memory corruption. (CVE-2013-2429, CVE-2013-2430)

The JDBC driver manager could incorrectly call the toString() method in
JDBC drivers, and the ConcurrentHashMap class could incorrectly call the
defaultReadObject() method. An untrusted Java application or applet could
possibly use these flaws to bypass Java sandbox restrictions.
(CVE-2013-1488, CVE-2013-2426)

The sun.awt.datatransfer.ClassLoaderObjectInputStream class may incorrectly
invoke the system class loader. An untrusted Java application or applet
could possibly use this flaw to bypass certain Java sandbox restrictions.
(CVE-2013-0401)

Flaws were discovered in the Network component's InetAddress serialization,
and the 2D component's font handling. An untrusted Java application or
applet could possibly use these flaws to crash the Java Virtual Machine.
(CVE-2013-2417, CVE-2013-2419)

The MBeanInstantiator class implementation in the OpenJDK JMX component did
not properly check class access before creating new instances. An untrusted
Java application or applet could use this flaw to create instances of
non-public classes. (CVE-2013-2424)

It was discovered that JAX-WS could possibly create temporary files with
insecure permissions. A local attacker could use this flaw to access
temporary files created by an application using JAX-WS. (CVE-2013-2415)

Note: If the web browser plug-in provided by the icedtea-web package was
installed, the issues exposed via Java applets could have been exploited
without user interaction if a user visited a malicious website.

This erratum also upgrades the OpenJDK package to IcedTea7 2.3.9. Refer to
the NEWS file, linked to in the References, for further information.

All users of java-1.7.0-openjdk are advised to upgrade to these updated
packages, which resolve these issues. All running instances of OpenJDK Java
must be restarted for the update to take effect.

'''

results = run_prompts(comments)

for res in results:
    print('Q: {}'.format(res['Q']))
    print('A: {}'.format(res['A']))
    

Q: what is the need to update java
A: java is a general purpose programming language
Q: Is there any open vulnerability in java
A: java.security.getpass.invalidate()
Q: Is there any fix for vulnerability in java 1.7
A: Java 1.7.25 is the latest update.
